In [1]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.Collecting jmespath<1.0.0,>=0.7.1



In [2]:
import boto3

In [3]:
s3 = boto3.resource('s3',
    aws_access_key_id='',
    aws_secret_access_key=''
)

In [8]:
try:
    s3.create_bucket(Bucket='datacont-benpixley', CreateBucketConfiguration={
        'LocationConstraint': 'us-west-2'})
except Exception as e:
    print (e)

In [9]:
bucket = s3.Bucket("datacont-benpixley")
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'K7MWR2K5ZP1AD17C',
  'HostId': 'LVNAMPSfU+zwvvQemcBVfUS64BaFDXm8MAjUFgQKJ+31kZ4X6fwOp7jJUB9sNFkimrljowsmt2A=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LVNAMPSfU+zwvvQemcBVfUS64BaFDXm8MAjUFgQKJ+31kZ4X6fwOp7jJUB9sNFkimrljowsmt2A=',
   'x-amz-request-id': 'K7MWR2K5ZP1AD17C',
   'date': 'Sun, 24 Oct 2021 19:15:08 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [11]:
body = open('exp1.csv','rb')

In [12]:
o = s3.Object('datacont-benpixley', 'test').put(Body=body )

In [13]:
s3.Object('datacont-benpixley', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'XJMGC1A3N773R617',
  'HostId': 'z1lJvRmQggZPMe6wXY12SbrVp9OcTgC1fkv8ypd5yi6Qbc+XFHVHP9T1V93rzcIfPEufbuWV7d8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'z1lJvRmQggZPMe6wXY12SbrVp9OcTgC1fkv8ypd5yi6Qbc+XFHVHP9T1V93rzcIfPEufbuWV7d8=',
   'x-amz-request-id': 'XJMGC1A3N773R617',
   'date': 'Sun, 24 Oct 2021 19:18:04 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [68]:
dyndb = boto3.resource('dynamodb',
region_name='us-west-2',
aws_access_key_id='',
aws_secret_access_key=''
)

In [82]:
try:
    table = dyndb.create_table(
        TableName='DataTable1',
        KeySchema=[
            {
                'AttributeName': 'PartitionKey',
                'KeyType': 'HASH'
            },
            {
                'AttributeName': 'RowKey',
                'KeyType': 'RANGE'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'PartitionKey',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'RowKey',
                'AttributeType': 'S'
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
except Exception as e:
    print (e)
    #if there is an exception, the table may already exist. if so...
    table = dyndb.Table("DataTable1")

An error occurred (ResourceInUseException) when calling the CreateTable operation: Table already exists: DataTable1


In [75]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable1')

In [81]:
print(table.item_count)

0


In [83]:
import csv
with open('Master.csv', 'r') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        if item[3] == 'Concentration':
            continue
        print(item)
        body = open(''+item[4], 'r')
        body = body.read().encode('utf-8');
        s3.Object('datacont-benpixley', item[4]).put(Body=body)
        md = s3.Object('datacont-benpixley', item[4]).Acl().put(ACL='public-read')
        url = " https://s3-us-west-2.amazonaws.com/datacont-benpixley/"+item[4]
        metadata_item = {'PartitionKey': item[4], 'RowKey': item[0], 'Temp': item[1], 
                         'Conductivity': item[2] , 'Concentration': item[3], 'url':url}
        table.put_item(Item=metadata_item)

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [85]:
response = table.get_item(
    Key={
        'PartitionKey': 'exp2.csv',
        'RowKey': '2'
    }
)
item = response['Item']
print(item)

{'Temp': '-2', 'RowKey': '2', 'Conductivity': '52.1', 'Concentration': '3.4', 'PartitionKey': 'exp2.csv', 'url': ' https://s3-us-west-2.amazonaws.com/datacont-benpixley/exp2.csv'}
